In [ ]:
import urllib.request, json
import xmltodict
import pandas as pd
with urllib.request.urlopen("https://www.fire.nsw.gov.au/feeds/stations-georss.php") as url:
    # print(url.read())
    o = xmltodict.parse(url.read().decode())
    strjsondata = json.dumps(o)
    jsondata = json.loads(strjsondata)
    df = pd.DataFrame.from_dict(jsondata['rss']['channel']['item'], orient="columns")
    df.to_csv("fire_station_NSW.csv")

In [1]:
import geopandas as gpd
import requests
from zipfile import ZipFile
import io
import pandas as pd

In [2]:
local_path = "./dataset/"
wildfire = pd.read_csv(f"{local_path}wildfire_data.csv")
wildfire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14334 entries, 0 to 14333
Data columns (total 58 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Fire_ID              14334 non-null  int64  
 1   Date                 14334 non-null  int64  
 2   AreaHa               14334 non-null  float64
 3   PerimeterM           14334 non-null  float64
 4   air_mean             14334 non-null  float64
 5   air_min              14334 non-null  float64
 6   air_max              14334 non-null  float64
 7   pre_mean             14334 non-null  float64
 8   pre_min              14334 non-null  float64
 9   pre_max              14334 non-null  float64
 10  soilw_mean           14334 non-null  float64
 11  soilw_min            14334 non-null  float64
 12  soilw_max            14334 non-null  float64
 13  air_mean_spring      14334 non-null  float64
 14  air_mean_summer      14334 non-null  float64
 15  air_mean_autumn      14334 non-null 

In [3]:
from ast import literal_eval
from shapely import Point

In [4]:
wildfire["geometry"] = wildfire["geometry"].apply(lambda c: Point(literal_eval(", ".join(c.strip("POINT ()").split(" ")))))

In [5]:
wildfire.head()

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,soilw_max_winter,wind_mean (m.s++-1),wind_min (m.s++-1),wind_max (m.s++-1),Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_mean,Classify,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.593708,5,POINT (150.0861388164024 -34.24668639480927)
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0672479394609 -34.13110194566074)
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0737348331977 -34.13389252032015)
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0760657232327 -34.14967444434212)
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.1532,Blue Mountains - South,Blue Mountains - South,0.769183,1,POINT (150.1331907652882 -34.22686843348224)


In [6]:
wildfire_gpd = gpd.GeoDataFrame(wildfire, crs="EPSG:4326", geometry = wildfire.geometry)

In [7]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json

import ipyleaflet
map = ipyleaflet.Map(center=(-27.47723161340788,153.02734507771476), zoom=15)
map


Map(center=[-27.47723161340788, 153.02734507771476], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [8]:
wildfire_gpd.head(10)

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,soilw_max_winter,wind_mean (m.s++-1),wind_min (m.s++-1),wind_max (m.s++-1),Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_mean,Classify,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.593708,5,POINT (150.08614 -34.24669)
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.06725 -34.13110)
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.07373 -34.13389)
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.07607 -34.14967)
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.15320,Blue Mountains - South,Blue Mountains - South,0.769183,1,POINT (150.13319 -34.22687)
5,63,1982,4161.875464,43699.485003,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.765850,5,POINT (150.17446 -34.00031)
6,64,1982,1.310912,433.266553,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.804033,5,POINT (150.12770 -34.08072)
7,123,1982,574.156662,11278.583394,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.323638,1.916126,2.745639,6.56977,The Oaks - Oakdale,Wollondilly,0.717108,5,POINT (150.46133 -34.05408)
8,212,1982,2.058188,525.972898,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.15320,Blue Mountains - South,Blue Mountains - South,0.813617,5,POINT (150.16990 -34.17413)
9,213,1982,4.345687,776.619843,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.802233,5,POINT (150.19992 -34.11703)


In [9]:
geo_data = GeoData(geo_dataframe = wildfire_gpd.tail(30),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

map.add_layer(geo_data)
map.add_control(LayersControl())

In [11]:
geo_gpd = gpd.read_file("./dataset/wildfire_sa2_level/wildfire_sa2_level.shp")
geo_gpd.head()

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,wind_min (,wind_max (,Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_,Classify,index_righ,SA2_CODE21,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.593708,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.148278,3.265098,14.1532,Blue Mountains - South,Blue Mountains - South,0.769183,1,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
